In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss
import xgboost as xgb

In [6]:
def read_train_test():
    X_train = pd.read_csv(
        "X_train.csv", index_col="client_id"
    )
    y_train = pd.read_csv(
        "y_train.csv",
        header=None,
        names=["client_id", "target"],
        index_col="client_id"
    )["target"]
    train_is_treatment = pd.read_csv(
        "X_train_is_treatment.csv",
        header=None,
        names=["client_id", "is_treatment"],
        index_col="client_id"
    )["is_treatment"]

    X_valid = pd.read_csv("X_valid.csv", index_col="client_id")
    y_valid = pd.read_csv(
        "y_valid.csv",
        header=None,
        names=["client_id", "target"],
        index_col="client_id"
    )["target"]
    valid_is_treatment = pd.read_csv(
        "X_valid_is_treatment.csv",
        header=None,
        names=["client_id", "is_treatment"],
        index_col="client_id"
    )["is_treatment"]

    X_test = pd.read_csv("X_test.csv", index_col="client_id")

    return X_train, y_train, train_is_treatment, X_valid, y_valid, valid_is_treatment, X_test


def join_train_validation(X_train, X_valid, y_train, y_valid):
    X_train = pd.concat([X_train, X_valid], ignore_index=False)
    y_train = pd.concat([y_train, y_valid], ignore_index=False)
    return X_train, y_train


def split_control_treatment(X, y, is_treatment):
    X_control = X[is_treatment == 0]
    X_treatment = X[is_treatment == 1]
    y_control = y[is_treatment == 0]
    y_treatment = y[is_treatment == 1]
    return X_control, X_treatment, y_control, y_treatment

In [7]:
def uplift_score(prediction, treatment, target, rate=0.3):
    """
    Подсчет Uplift Score
    """
    try:
        order = np.argsort(-prediction.values)
    except Exception:
        order = np.argsort(-prediction)
    treatment_n = int((treatment == 1).sum() * rate)
#     print(f"    number of treatment users: {treatment_n}")
    treatment_p = target[order][treatment[order] == 1][:treatment_n].mean()
#     print(f"    treatment p: {treatment_p}")
    control_n = int((treatment == 0).sum() * rate)
#     print(f"    number of control users: {treatment_n}")
    control_p = target[order][treatment[order] == 0][:control_n].mean()
#     print(f"    control p: {control_p}")
    score = treatment_p - control_p
    return score

# Baseline (no custom features, xgb without tuning)

In [8]:
X_train, y_train, train_is_treatment, X_valid, y_valid, valid_is_treatment, X_test = read_train_test()

In [9]:
X_train_control, X_train_treatment, y_train_control, y_train_treatment = split_control_treatment(
    X_train, y_train, train_is_treatment
)
X_valid_control, X_valid_treatment, y_valid_control, y_valid_treatment = split_control_treatment(
    X_valid, y_valid, valid_is_treatment
)

In [6]:
baseline_cols = [
    'age', 
    'strange_age',
    'gender_F',
    'gender_M',
    'gender_U',
    'redeem_issue_diff',
    'first_issue_dayofyear',
    'first_issue_hour',
    'first_issue_weekday',
    'first_issue_dayofmonth',
    'first_issue_year',
    'first_issue_month',
    'first_issue_weekofyear',
    'first_issue_week',
    'first_issue_quarter',
    'diff'
]

## Hold-out set

In [16]:
X_train_baseline_cols = X_train[baseline_cols]
X_valid_baseline_cols = X_valid[baseline_cols]

X_train_control_, X_train_treatment_, y_train_control_, y_train_treatment_ = split_control_treatment(
    X_train_baseline_cols, y_train, train_is_treatment
)

xgb_control = xgb.XGBClassifier().fit(X_train_control_, y_train_control_)
control_preds = xgb_control.predict(X_valid_baseline_cols)
control_proba = xgb_control.predict_proba(X_valid_baseline_cols)[:, 1]

acc = accuracy_score(y_valid, control_preds)
auc = roc_auc_score(y_valid, control_proba)
logloss = log_loss(y_valid, control_proba)
print(f"Control accuracy score: {acc}")
print(f"Control roc auc score: {auc}")
print(f"Control log loss: {logloss}\n")

xgb_treatment = xgb.XGBClassifier().fit(X_train_treatment_, y_train_treatment_)
treatment_preds = xgb_treatment.predict(X_valid_baseline_cols)
treatment_proba = xgb_treatment.predict_proba(X_valid_baseline_cols)[:, 1]
acc = accuracy_score(y_valid, treatment_preds)
auc = roc_auc_score(y_valid, treatment_proba)
logloss = log_loss(y_valid, treatment_proba)
print(f"Treatment accuracy score: {acc}")
print(f"Treatment roc auc score: {auc}")
print(f"Treatment log loss: {logloss}\n")

uplift = treatment_proba - control_proba
up_score = uplift_score(uplift, valid_is_treatment, y_valid)
uplifts.append(up_score)
print(f"Uplift score: {up_score}\n")

Control accuracy score: 0.6459208158368326
Control roc auc score: 0.5998777443668605
Control log loss: 0.6452512383476642

Treatment accuracy score: 0.6455708858228354
Treatment roc auc score: 0.601243738502897
Treatment log loss: 0.6448943353969647

Uplift score: 0.05122098384721485



## 1 K-fold CV (K=5)

In [9]:
X_train_baseline_cols = X_train[baseline_cols]

control_acc = []
control_auc = []
control_loglos = []

treatment_acc = []
treatment_auc = []
treatment_loglos = []

uplifts = []

skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=142)
for i, (train_idx, test_idx) in enumerate(skfold.split(X_train_baseline_cols, y_train)):
    print(f"\nFold {i + 1}")

    X_train_, y_train_ = X_train_baseline_cols.iloc[train_idx], y_train.iloc[train_idx]
    X_test_, y_test_ = X_train_baseline_cols.iloc[test_idx], y_train.iloc[test_idx]
    train_is_treatment_, test_is_treatment_ = train_is_treatment.iloc[train_idx], train_is_treatment.iloc[test_idx]

    X_train_control_, X_train_treatment_, y_train_control_, y_train_treatment_ = split_control_treatment(
        X_train_, y_train_, train_is_treatment_
    )

    xgb_control = xgb.XGBClassifier().fit(X_train_control_, y_train_control_)
    control_preds = xgb_control.predict(X_test_)
    control_proba = xgb_control.predict_proba(X_test_)[:, 1]
    acc = accuracy_score(y_test_, control_preds)
    auc = roc_auc_score(y_test_, control_proba)
    logloss = log_loss(y_test_, control_proba)
    control_acc.append(acc)
    control_auc.append(auc)
    control_loglos.append(logloss)
    print(f"Control accuracy score: {acc}")
    print(f"Control roc auc score: {auc}")
    print(f"Control log loss: {logloss}\n")

    xgb_treatment = xgb.XGBClassifier().fit(X_train_treatment_, y_train_treatment_)
    treatment_preds = xgb_treatment.predict(X_test_)
    treatment_proba = xgb_treatment.predict_proba(X_test_)[:, 1]
    acc = accuracy_score(y_test_, treatment_preds)
    auc = roc_auc_score(y_test_, treatment_proba)
    logloss = log_loss(y_test_, treatment_proba)
    treatment_acc.append(acc)
    treatment_auc.append(auc)
    treatment_loglos.append(logloss)
    print(f"Control accuracy score: {acc}")
    print(f"Control roc auc score: {auc}")
    print(f"Control log loss: {logloss}\n")

    uplift = treatment_proba - control_proba
    up_score = uplift_score(uplift, test_is_treatment_, y_test_)
    uplifts.append(up_score)
    print(f"Uplift score: {up_score}\n")

print(f"\nControl average accuracy: {np.mean(control_acc)}")
print(f"Control average rocauc: {np.mean(control_auc)}")
print(f"Control average loglos: {np.mean(control_loglos)}\n")

print(f"Treatment average accuracy: {np.mean(treatment_acc)}")
print(f"Treatment average rocauc: {np.mean(treatment_auc)}")
print(f"Treatment average loglos: {np.mean(treatment_loglos)}\n")

print(f"Average uplift score: {np.mean(uplifts)}\n\n")


Fold 1
Control accuracy score: 0.6454525572531009
Control roc auc score: 0.6018141863003091
Control log loss: 0.6448891778220857

Control accuracy score: 0.6459212047364639
Control roc auc score: 0.605512802617282
Control log loss: 0.6438640065787528

Uplift score: 0.03449894908335316


Fold 2
Control accuracy score: 0.6429843471740557
Control roc auc score: 0.5977008429527652
Control log loss: 0.6467991020340398

Control accuracy score: 0.6417033773861968
Control roc auc score: 0.5985142158842853
Control log loss: 0.647036450061057

Uplift score: 0.05615416102931936


Fold 3
Control accuracy score: 0.6427232393926139
Control roc auc score: 0.6028354718419726
Control log loss: 0.6458837837590722

Control accuracy score: 0.642660751109167
Control roc auc score: 0.6032445732855591
Control log loss: 0.6454938918599813

Uplift score: 0.05743000549619892


Fold 4
Control accuracy score: 0.6460038742735738
Control roc auc score: 0.6004235453624058
Control log loss: 0.6450564669239024

Contr

## Average of 6 different K-folds

In [7]:
X_train_baseline_cols = X_train[baseline_cols]

seeds = [int(i + "42") for i in ["", "1", "2", "3", "4", "5"]]
kfolds_uplifts = []
for seed in seeds:
    print(f"SEED {seed}")
    control_acc = []
    control_auc = []
    control_loglos = []

    treatment_acc = []
    treatment_auc = []
    treatment_loglos = []

    uplifts = []
    
    skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=int(seed))
    for i, (train_idx, test_idx) in enumerate(skfold.split(X_train_baseline_cols, y_train)):
        print(f"\nFold {i + 1}")

        X_train_, y_train_ = X_train_baseline_cols.iloc[train_idx], y_train.iloc[train_idx]
        X_test_, y_test_ = X_train_baseline_cols.iloc[test_idx], y_train.iloc[test_idx]
        train_is_treatment_, test_is_treatment_ = train_is_treatment.iloc[train_idx], train_is_treatment.iloc[test_idx]

        X_train_control_, X_train_treatment_, y_train_control_, y_train_treatment_ = split_control_treatment(
            X_train_, y_train_, train_is_treatment_
        )

        xgb_control = xgb.XGBClassifier().fit(X_train_control_, y_train_control_)
        control_preds = xgb_control.predict(X_test_)
        control_proba = xgb_control.predict_proba(X_test_)[:, 1]
        acc = accuracy_score(y_test_, control_preds)
        auc = roc_auc_score(y_test_, control_proba)
        logloss = log_loss(y_test_, control_proba)
        control_acc.append(acc)
        control_auc.append(auc)
        control_loglos.append(logloss)
        print(f"Control accuracy score: {acc}")
        print(f"Control roc auc score: {auc}")
        print(f"Control log loss: {logloss}\n")

        xgb_treatment = xgb.XGBClassifier().fit(X_train_treatment_, y_train_treatment_)
        treatment_preds = xgb_treatment.predict(X_test_)
        treatment_proba = xgb_treatment.predict_proba(X_test_)[:, 1]
        acc = accuracy_score(y_test_, treatment_preds)
        auc = roc_auc_score(y_test_, treatment_proba)
        logloss = log_loss(y_test_, treatment_proba)
        treatment_acc.append(acc)
        treatment_auc.append(auc)
        treatment_loglos.append(logloss)
        print(f"Control accuracy score: {acc}")
        print(f"Control roc auc score: {auc}")
        print(f"Control log loss: {logloss}\n")

        uplift = treatment_proba - control_proba
        up_score = uplift_score(uplift, test_is_treatment_, y_test_)
        uplifts.append(up_score)
        print(f"Uplift score: {up_score}\n")

    print(f"\nControl average accuracy: {np.mean(control_acc)}")
    print(f"Control average rocauc: {np.mean(control_auc)}")
    print(f"Control average loglos: {np.mean(control_loglos)}\n")

    print(f"Treatment average accuracy: {np.mean(treatment_acc)}")
    print(f"Treatment average rocauc: {np.mean(treatment_auc)}")
    print(f"Treatment average loglos: {np.mean(treatment_loglos)}\n")

    print(f"Average uplift score: {np.mean(uplifts)}\n\n")
    kfolds_uplifts.append(np.mean(uplifts))
    
print(f"Average for uplift score all kfolds: {np.mean(kfolds_uplifts)}")


SEED 42

Fold 1
Control accuracy score: 0.6446714781141625
Control roc auc score: 0.6008771663833372
Control log loss: 0.6455221249061054

Control accuracy score: 0.6447652076108351
Control roc auc score: 0.6010536549168206
Control log loss: 0.6452335429859388

Uplift score: 0.05441349281565799


Fold 2
Control accuracy score: 0.644640234948605
Control roc auc score: 0.6011751602371477
Control log loss: 0.6451905292706189

Control accuracy score: 0.6450776392664105
Control roc auc score: 0.6026909351210843
Control log loss: 0.6448732524417492

Uplift score: 0.06140763807163163


Fold 3
Control accuracy score: 0.644410423045679
Control roc auc score: 0.6039763139947711
Control log loss: 0.644976954686264

Control accuracy score: 0.6431919015184653
Control roc auc score: 0.6086822617243062
Control log loss: 0.6442083537481743

Uplift score: 0.031003827298618836


Fold 4
Control accuracy score: 0.6430356808098482
Control roc auc score: 0.5998687349947461
Control log loss: 0.64619792048066

## Fit on whole set, generate submission

In [18]:
X_train_control_baseline_cols, X_train_treatment_baseline_cols, y_train_control, y_train_treatment = split_control_treatment(
    X_train_baseline_cols, y_train, train_is_treatment
)
X_valid_control_baseline_cols, X_valid_treatment_baseline_cols, y_valid_control, y_valid_treatment = split_control_treatment(
    X_valid_baseline_cols, y_valid, valid_is_treatment
)

In [19]:
X_control_baseline_cols, y_control = join_train_validation(
    X_train_control_baseline_cols, X_valid_control_baseline_cols, y_train_control, y_valid_control
)
X_treatment_baseline_cols, y_treatment = join_train_validation(
    X_train_treatment_baseline_cols, X_valid_treatment_baseline_cols, y_train_treatment, y_valid_treatment
)
X_test_baseline_cols = X_test[baseline_cols]

clf_control = xgb.XGBClassifier().fit(X_control_baseline_cols, y_control)
clf_treatment = xgb.XGBClassifier().fit(X_treatment_baseline_cols, y_treatment)

predict_test_control = clf_control.predict_proba(X_test_baseline_cols)[:, 1]
predict_test_treatment = clf_treatment.predict_proba(X_test_baseline_cols)[:, 1]
predict_test_uplift = predict_test_treatment - predict_test_control
df_submission = pd.DataFrame({'uplift': predict_test_uplift}, index=X_test.index)
df_submission.to_csv('basline_cols_xgb_no_tuning.csv')

# All features

In [21]:
X_train, y_train, train_is_treatment, X_valid, y_valid, valid_is_treatment, X_test = read_train_test()

In [22]:
X_train_control, X_train_treatment, y_train_control, y_train_treatment = split_control_treatment(
    X_train, y_train, train_is_treatment
)
X_valid_control, X_valid_treatment, y_valid_control, y_valid_treatment = split_control_treatment(
    X_valid, y_valid, valid_is_treatment
)

## Hold-out set

In [23]:
xgb_control = xgb.XGBClassifier().fit(X_train_control, y_train_control)
control_preds = xgb_control.predict(X_valid)
control_proba = xgb_control.predict_proba(X_valid)[:, 1]

acc = accuracy_score(y_valid, control_preds)
auc = roc_auc_score(y_valid, control_proba)
logloss = log_loss(y_valid, control_proba)
print(f"Control accuracy score: {acc}")
print(f"Control roc auc score: {auc}")
print(f"Control log loss: {logloss}\n")

xgb_treatment = xgb.XGBClassifier().fit(X_train_treatment, y_train_treatment)
treatment_preds = xgb_treatment.predict(X_valid)
treatment_proba = xgb_treatment.predict_proba(X_valid)[:, 1]
acc = accuracy_score(y_valid, treatment_preds)
auc = roc_auc_score(y_valid, treatment_proba)
logloss = log_loss(y_valid, treatment_proba)
print(f"Treatment accuracy score: {acc}")
print(f"Treatment roc auc score: {auc}")
print(f"Treatment log loss: {logloss}\n")

uplift = treatment_proba - control_proba
up_score = uplift_score(uplift, valid_is_treatment, y_valid)
uplifts.append(up_score)
print(f"Uplift score: {up_score}\n")

Control accuracy score: 0.7156068786242752
Control roc auc score: 0.7741321502293599
Control log loss: 0.5481313662264564

Treatment accuracy score: 0.7146320735852829
Treatment roc auc score: 0.7746902210186017
Treatment log loss: 0.5472975145549888

Uplift score: 0.05746852762817389



## 1 K-fold (K=5)

In [25]:
control_acc = []
control_auc = []
control_loglos = []

treatment_acc = []
treatment_auc = []
treatment_loglos = []

uplifts = []

skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=142)
for i, (train_idx, test_idx) in enumerate(skfold.split(X_train, y_train)):
    print(f"\nFold {i + 1}")

    X_train_, y_train_ = X_train.iloc[train_idx], y_train.iloc[train_idx]
    X_test_, y_test_ = X_train.iloc[test_idx], y_train.iloc[test_idx]
    train_is_treatment_, test_is_treatment_ = train_is_treatment.iloc[train_idx], train_is_treatment.iloc[test_idx]

    X_train_control_, X_train_treatment_, y_train_control_, y_train_treatment_ = split_control_treatment(
        X_train_, y_train_, train_is_treatment_
    )

    xgb_control = xgb.XGBClassifier().fit(X_train_control_, y_train_control_)
    control_preds = xgb_control.predict(X_test_)
    control_proba = xgb_control.predict_proba(X_test_)[:, 1]
    acc = accuracy_score(y_test_, control_preds)
    auc = roc_auc_score(y_test_, control_proba)
    logloss = log_loss(y_test_, control_proba)
    control_acc.append(acc)
    control_auc.append(auc)
    control_loglos.append(logloss)
    print(f"Control accuracy score: {acc}")
    print(f"Control roc auc score: {auc}")
    print(f"Control log loss: {logloss}\n")

    xgb_treatment = xgb.XGBClassifier().fit(X_train_treatment_, y_train_treatment_)
    treatment_preds = xgb_treatment.predict(X_test_)
    treatment_proba = xgb_treatment.predict_proba(X_test_)[:, 1]
    acc = accuracy_score(y_test_, treatment_preds)
    auc = roc_auc_score(y_test_, treatment_proba)
    logloss = log_loss(y_test_, treatment_proba)
    treatment_acc.append(acc)
    treatment_auc.append(auc)
    treatment_loglos.append(logloss)
    print(f"Control accuracy score: {acc}")
    print(f"Control roc auc score: {auc}")
    print(f"Control log loss: {logloss}\n")

    uplift = treatment_proba - control_proba
    up_score = uplift_score(uplift, test_is_treatment_, y_test_)
    uplifts.append(up_score)
    print(f"Uplift score: {up_score}\n")

print(f"\nControl average accuracy: {np.mean(control_acc)}")
print(f"Control average rocauc: {np.mean(control_auc)}")
print(f"Control average loglos: {np.mean(control_loglos)}\n")

print(f"Treatment average accuracy: {np.mean(treatment_acc)}")
print(f"Treatment average rocauc: {np.mean(treatment_auc)}")
print(f"Treatment average loglos: {np.mean(treatment_loglos)}\n")

print(f"Average uplift score: {np.mean(uplifts)}\n\n")


Fold 1
Control accuracy score: 0.7114381229106133
Control roc auc score: 0.7673465885917468
Control log loss: 0.5537475528253172

Control accuracy score: 0.7137188739963133
Control roc auc score: 0.7688245653645973
Control log loss: 0.5524289729941039

Uplift score: 0.058050191619362757


Fold 2
Control accuracy score: 0.7130627675196051
Control roc auc score: 0.7696804319147839
Control log loss: 0.5521839235294425

Control accuracy score: 0.7140313056518887
Control roc auc score: 0.7705581832171067
Control log loss: 0.551071622950634

Uplift score: 0.06582438895928122


Fold 3
Control accuracy score: 0.712053989876898
Control roc auc score: 0.7701017811167233
Control log loss: 0.5519786537779408

Control accuracy score: 0.7111479097669187
Control roc auc score: 0.7704456516919196
Control log loss: 0.5510462282718649

Uplift score: 0.07520690825722831


Fold 4
Control accuracy score: 0.7155220896081984
Control roc auc score: 0.7745385282689854
Control log loss: 0.5474970890734595

Con

## 6 K-folds with different seeds

In [8]:
seeds = [int(i + "42") for i in ["", "1", "2", "3", "4", "5"]]
kfolds_uplifts = []
for seed in seeds:
    print(f"SEED {seed}")
    control_acc = []
    control_auc = []
    control_loglos = []

    treatment_acc = []
    treatment_auc = []
    treatment_loglos = []

    uplifts = []
    
    skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=int(seed))
    for i, (train_idx, test_idx) in enumerate(skfold.split(X_train, y_train)):
        print(f"\nFold {i + 1}")

        X_train_, y_train_ = X_train.iloc[train_idx], y_train.iloc[train_idx]
        X_test_, y_test_ = X_train.iloc[test_idx], y_train.iloc[test_idx]
        train_is_treatment_, test_is_treatment_ = train_is_treatment.iloc[train_idx], train_is_treatment.iloc[test_idx]

        X_train_control_, X_train_treatment_, y_train_control_, y_train_treatment_ = split_control_treatment(
            X_train_, y_train_, train_is_treatment_
        )

        xgb_control = xgb.XGBClassifier().fit(X_train_control_, y_train_control_)
        control_preds = xgb_control.predict(X_test_)
        control_proba = xgb_control.predict_proba(X_test_)[:, 1]
        acc = accuracy_score(y_test_, control_preds)
        auc = roc_auc_score(y_test_, control_proba)
        logloss = log_loss(y_test_, control_proba)
        control_acc.append(acc)
        control_auc.append(auc)
        control_loglos.append(logloss)
        print(f"Control accuracy score: {acc}")
        print(f"Control roc auc score: {auc}")
        print(f"Control log loss: {logloss}\n")

        xgb_treatment = xgb.XGBClassifier().fit(X_train_treatment_, y_train_treatment_)
        treatment_preds = xgb_treatment.predict(X_test_)
        treatment_proba = xgb_treatment.predict_proba(X_test_)[:, 1]
        acc = accuracy_score(y_test_, treatment_preds)
        auc = roc_auc_score(y_test_, treatment_proba)
        logloss = log_loss(y_test_, treatment_proba)
        treatment_acc.append(acc)
        treatment_auc.append(auc)
        treatment_loglos.append(logloss)
        print(f"Control accuracy score: {acc}")
        print(f"Control roc auc score: {auc}")
        print(f"Control log loss: {logloss}\n")

        uplift = treatment_proba - control_proba
        up_score = uplift_score(uplift, test_is_treatment_, y_test_)
        uplifts.append(up_score)
        print(f"Uplift score: {up_score}\n")

    print(f"\nControl average accuracy: {np.mean(control_acc)}")
    print(f"Control average rocauc: {np.mean(control_auc)}")
    print(f"Control average loglos: {np.mean(control_loglos)}\n")

    print(f"Treatment average accuracy: {np.mean(treatment_acc)}")
    print(f"Treatment average rocauc: {np.mean(treatment_auc)}")
    print(f"Treatment average loglos: {np.mean(treatment_loglos)}\n")

    print(f"Average uplift score: {np.mean(uplifts)}\n\n")
    kfolds_uplifts.append(np.mean(uplifts))
    
print(f"Average for uplift score all kfolds: {np.mean(kfolds_uplifts)}")


SEED 42

Fold 1
Control accuracy score: 0.7118442840628613
Control roc auc score: 0.7712134747505055
Control log loss: 0.5505524108073874

Control accuracy score: 0.7112506639172681
Control roc auc score: 0.7707821529220812
Control log loss: 0.5506969216692374

Uplift score: 0.06310895065907096


Fold 2
Control accuracy score: 0.7109382322616927
Control roc auc score: 0.7684337052655865
Control log loss: 0.5530596066437508

Control accuracy score: 0.7123129315462242
Control roc auc score: 0.7694246727742813
Control log loss: 0.5517340281536958

Uplift score: 0.06788882061536428


Fold 3
Control accuracy score: 0.715709554458539
Control roc auc score: 0.775458500590483
Control log loss: 0.5464467912894794

Control accuracy score: 0.716803099418859
Control roc auc score: 0.7768160512379068
Control log loss: 0.5454247755656186

Uplift score: 0.04645105037158337


Fold 4
Control accuracy score: 0.7138973942385802
Control roc auc score: 0.7698967405212994
Control log loss: 0.551654175063045

## Fit on whole training set and generate submission

In [27]:
X_train, y_train, train_is_treatment, X_valid, y_valid, valid_is_treatment, X_test = read_train_test()

In [28]:
X_train_control, X_train_treatment, y_train_control, y_train_treatment = split_control_treatment(
    X_train, y_train, train_is_treatment
)
X_valid_control, X_valid_treatment, y_valid_control, y_valid_treatment = split_control_treatment(
    X_valid, y_valid, valid_is_treatment
)

In [29]:
X_control, y_control = join_train_validation(X_train_control, X_valid_control, y_train_control, y_valid_control)
X_treatment, y_treatment = join_train_validation(
    X_train_treatment, X_valid_treatment, y_train_treatment, y_valid_treatment
)

clf_control = xgb.XGBClassifier().fit(X_control, y_control)
clf_treatment = xgb.XGBClassifier().fit(X_treatment, y_treatment)

predict_test_control = clf_control.predict_proba(X_test)[:, 1]
predict_test_treatment = clf_treatment.predict_proba(X_test)[:, 1]
predict_test_uplift = predict_test_treatment - predict_test_control
df_submission = pd.DataFrame({'uplift': predict_test_uplift}, index=X_test.index)
df_submission.to_csv('all_features_xgb_no_tuning.csv')

In [31]:
uplift_holdout_baseline_cols = 0.05122098384721485
uplift_1_kfold_baseline_cols = 0.04775477592059265
uplift_6_kfolds_baseline_cols = 0.04856602992351957

uplift_holdout_all_features = 0.05746852762817389
uplift_1_kfold_all_features = 0.05989159396417358
uplift_6_kfolds_all_features = 0.058104041182405114

holdout_diff = uplift_holdout_all_features - uplift_holdout_baseline_cols
kfold_diff = uplift_1_kfold_all_features - uplift_1_kfold_baseline_cols
kfolds_diff = uplift_6_kfolds_all_features - uplift_6_kfolds_baseline_cols

baseline_score = 0.0585
all_features_score = 0.0566
real_diff = all_features_score - baseline_score

print(f"holdout diff: {holdout_diff}")
print(f"1 kfold diff: {kfold_diff}")
print(f"6 kfolds diff: {kfolds_diff}")
print(f"real diff: {real_diff}")

holdout diff: 0.006247543780959042
1 kfold diff: 0.012136818043580931
6 kfolds diff: 0.009538011258885544
real diff: -0.0019000000000000059


...

# Tuned XGBoost

In [10]:
xgb_best_params = {
    "subsample": 0.7, 
    "reg_lambda": 1.0, 
    "reg_alpha": 0, 
    "n_estimators": 1000, 
    "min_child_weight": 7, 
    "max_depth": 4, 
    "learning_rate": 0.01,
    "gamma": 0.3, 
    "colsample_bytree": 0.5
}

## Hold-out set

In [34]:
xgb_control = xgb.XGBClassifier(objective="binary:logistic", **xgb_best_params).fit(X_train_control, y_train_control)
control_preds = xgb_control.predict(X_valid)
control_proba = xgb_control.predict_proba(X_valid)[:, 1]

acc = accuracy_score(y_valid, control_preds)
auc = roc_auc_score(y_valid, control_proba)
logloss = log_loss(y_valid, control_proba)
print(f"Control accuracy score: {acc}")
print(f"Control roc auc score: {auc}")
print(f"Control log loss: {logloss}\n")

xgb_treatment = xgb.XGBClassifier(objective="binary:logistic", **xgb_best_params).fit(X_train_treatment, y_train_treatment)
treatment_preds = xgb_treatment.predict(X_valid)
treatment_proba = xgb_treatment.predict_proba(X_valid)[:, 1]
acc = accuracy_score(y_valid, treatment_preds)
auc = roc_auc_score(y_valid, treatment_proba)
logloss = log_loss(y_valid, treatment_proba)
print(f"Treatment accuracy score: {acc}")
print(f"Treatment roc auc score: {auc}")
print(f"Treatment log loss: {logloss}\n")

uplift = treatment_proba - control_proba
up_score = uplift_score(uplift, valid_is_treatment, y_valid)
uplifts.append(up_score)
print(f"Uplift score: {up_score}\n")

Control accuracy score: 0.716006798640272
Control roc auc score: 0.7746608757415683
Control log loss: 0.5475571413952864

Treatment accuracy score: 0.7155818836232753
Treatment roc auc score: 0.775409262520435
Treatment log loss: 0.5464728559200964

Uplift score: 0.08186243440548335



## 1 K-fold (K=5)

In [35]:
control_acc = []
control_auc = []
control_loglos = []

treatment_acc = []
treatment_auc = []
treatment_loglos = []

uplifts = []

skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=142)
for i, (train_idx, test_idx) in enumerate(skfold.split(X_train, y_train)):
    print(f"\nFold {i + 1}")

    X_train_, y_train_ = X_train.iloc[train_idx], y_train.iloc[train_idx]
    X_test_, y_test_ = X_train.iloc[test_idx], y_train.iloc[test_idx]
    train_is_treatment_, test_is_treatment_ = train_is_treatment.iloc[train_idx], train_is_treatment.iloc[test_idx]

    X_train_control_, X_train_treatment_, y_train_control_, y_train_treatment_ = split_control_treatment(
        X_train_, y_train_, train_is_treatment_
    )

    xgb_control = xgb.XGBClassifier(objective="binary:logistic", **xgb_best_params).fit(X_train_control_, y_train_control_)
    control_preds = xgb_control.predict(X_test_)
    control_proba = xgb_control.predict_proba(X_test_)[:, 1]
    acc = accuracy_score(y_test_, control_preds)
    auc = roc_auc_score(y_test_, control_proba)
    logloss = log_loss(y_test_, control_proba)
    control_acc.append(acc)
    control_auc.append(auc)
    control_loglos.append(logloss)
    print(f"Control accuracy score: {acc}")
    print(f"Control roc auc score: {auc}")
    print(f"Control log loss: {logloss}\n")

    xgb_treatment = xgb.XGBClassifier(objective="binary:logistic", **xgb_best_params).fit(X_train_treatment_, y_train_treatment_)
    treatment_preds = xgb_treatment.predict(X_test_)
    treatment_proba = xgb_treatment.predict_proba(X_test_)[:, 1]
    acc = accuracy_score(y_test_, treatment_preds)
    auc = roc_auc_score(y_test_, treatment_proba)
    logloss = log_loss(y_test_, treatment_proba)
    treatment_acc.append(acc)
    treatment_auc.append(auc)
    treatment_loglos.append(logloss)
    print(f"Control accuracy score: {acc}")
    print(f"Control roc auc score: {auc}")
    print(f"Control log loss: {logloss}\n")

    uplift = treatment_proba - control_proba
    up_score = uplift_score(uplift, test_is_treatment_, y_test_)
    uplifts.append(up_score)
    print(f"Uplift score: {up_score}\n")

print(f"\nControl average accuracy: {np.mean(control_acc)}")
print(f"Control average rocauc: {np.mean(control_auc)}")
print(f"Control average loglos: {np.mean(control_loglos)}\n")

print(f"Treatment average accuracy: {np.mean(treatment_acc)}")
print(f"Treatment average rocauc: {np.mean(treatment_auc)}")
print(f"Treatment average loglos: {np.mean(treatment_loglos)}\n")

print(f"Average uplift score: {np.mean(uplifts)}\n\n")


Fold 1
Control accuracy score: 0.7100946667916394
Control roc auc score: 0.7681524926214873
Control log loss: 0.5531099352925214

Control accuracy score: 0.7130627675196051
Control roc auc score: 0.7693160319434034
Control log loss: 0.5517533203449537

Uplift score: 0.0536815843195495


Fold 2
Control accuracy score: 0.7125941200362421
Control roc auc score: 0.7697720574855065
Control log loss: 0.5519309507853838

Control accuracy score: 0.715218545943075
Control roc auc score: 0.7711496576865198
Control log loss: 0.5504999132654959

Uplift score: 0.05243055081031378


Fold 3
Control accuracy score: 0.7121477223020684
Control roc auc score: 0.7710165709273386
Control log loss: 0.5511110204550211

Control accuracy score: 0.7104605386490033
Control roc auc score: 0.7713093061523522
Control log loss: 0.5501462862863722

Uplift score: 0.07254318539690807


Fold 4
Control accuracy score: 0.7152096481909642
Control roc auc score: 0.7748732724880871
Control log loss: 0.5471449461199624

Cont

## 6 K-folds with different seeds

In [11]:
seeds = [int(i + "42") for i in ["", "1", "2", "3", "4", "5"]]
kfolds_uplifts = []
for seed in seeds:
    print(f"SEED {seed}")
    control_acc = []
    control_auc = []
    control_loglos = []

    treatment_acc = []
    treatment_auc = []
    treatment_loglos = []

    uplifts = []
    
    skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=int(seed))
    for i, (train_idx, test_idx) in enumerate(skfold.split(X_train, y_train)):
        print(f"\nFold {i + 1}")

        X_train_, y_train_ = X_train.iloc[train_idx], y_train.iloc[train_idx]
        X_test_, y_test_ = X_train.iloc[test_idx], y_train.iloc[test_idx]
        train_is_treatment_, test_is_treatment_ = train_is_treatment.iloc[train_idx], train_is_treatment.iloc[test_idx]

        X_train_control_, X_train_treatment_, y_train_control_, y_train_treatment_ = split_control_treatment(
            X_train_, y_train_, train_is_treatment_
        )

        xgb_control = xgb.XGBClassifier(objective="binary:logistic", **xgb_best_params).fit(X_train_control_, y_train_control_)
        control_preds = xgb_control.predict(X_test_)
        control_proba = xgb_control.predict_proba(X_test_)[:, 1]
        acc = accuracy_score(y_test_, control_preds)
        auc = roc_auc_score(y_test_, control_proba)
        logloss = log_loss(y_test_, control_proba)
        control_acc.append(acc)
        control_auc.append(auc)
        control_loglos.append(logloss)
        print(f"Control accuracy score: {acc}")
        print(f"Control roc auc score: {auc}")
        print(f"Control log loss: {logloss}\n")

        xgb_treatment = xgb.XGBClassifier(objective="binary:logistic", **xgb_best_params).fit(X_train_treatment_, y_train_treatment_)
        treatment_preds = xgb_treatment.predict(X_test_)
        treatment_proba = xgb_treatment.predict_proba(X_test_)[:, 1]
        acc = accuracy_score(y_test_, treatment_preds)
        auc = roc_auc_score(y_test_, treatment_proba)
        logloss = log_loss(y_test_, treatment_proba)
        treatment_acc.append(acc)
        treatment_auc.append(auc)
        treatment_loglos.append(logloss)
        print(f"Control accuracy score: {acc}")
        print(f"Control roc auc score: {auc}")
        print(f"Control log loss: {logloss}\n")

        uplift = treatment_proba - control_proba
        up_score = uplift_score(uplift, test_is_treatment_, y_test_)
        uplifts.append(up_score)
        print(f"Uplift score: {up_score}\n")

    print(f"\nControl average accuracy: {np.mean(control_acc)}")
    print(f"Control average rocauc: {np.mean(control_auc)}")
    print(f"Control average loglos: {np.mean(control_loglos)}\n")

    print(f"Treatment average accuracy: {np.mean(treatment_acc)}")
    print(f"Treatment average rocauc: {np.mean(treatment_auc)}")
    print(f"Treatment average loglos: {np.mean(treatment_loglos)}\n")

    print(f"Average uplift score: {np.mean(uplifts)}\n\n")
    kfolds_uplifts.append(np.mean(uplifts))
    
print(f"Average for uplift score all kfolds: {np.mean(kfolds_uplifts)}")


SEED 42

Fold 1


/home/kbrodt/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Control accuracy score: 0.713875089824101
Control roc auc score: 0.7709506630280397
Control log loss: 0.5508473347834087



/home/kbrodt/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Control accuracy score: 0.7159371387508983
Control roc auc score: 0.7729311668166081
Control log loss: 0.5492609231459415

Uplift score: 0.06457404434932534


Fold 2


...

Control accuracy score: 0.713147534837218
Control roc auc score: 0.7717467374126215
Control log loss: 0.5501751066391259

Uplift score: 0.06458386671515204


Fold 5


/home/kbrodt/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Control accuracy score: 0.7100765505389783
Control roc auc score: 0.7666618430690748
Control log loss: 0.5546183927405735



/home/kbrodt/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Control accuracy score: 0.7128886111545071
Control roc auc score: 0.7686673475385686
Control log loss: 0.5523552555458175

Uplift score: 0.05479061909384164


Control average accuracy: 0.7131618019742321
Control average rocauc: 0.7712095856716694
Control average loglos: 0.5505403906702326

Treatment average accuracy: 0.7140303924342053
Treatment average rocauc: 0.7721408445306812
Treatment average loglos: 0.5494153585976462

Average uplift score: 0.062266248268947035


Average for uplift score all kfolds: 0.06014021157990412


## Fit on whole training set and generate submission

In [12]:
X_train, y_train, train_is_treatment, X_valid, y_valid, valid_is_treatment, X_test = read_train_test()

In [13]:
X_train_control, X_train_treatment, y_train_control, y_train_treatment = split_control_treatment(
    X_train, y_train, train_is_treatment
)
X_valid_control, X_valid_treatment, y_valid_control, y_valid_treatment = split_control_treatment(
    X_valid, y_valid, valid_is_treatment
)

In [15]:
X_control, y_control = join_train_validation(X_train_control, X_valid_control, y_train_control, y_valid_control)
X_treatment, y_treatment = join_train_validation(
    X_train_treatment, X_valid_treatment, y_train_treatment, y_valid_treatment
)

clf_control = xgb.XGBClassifier(objective="binary:logistic", **xgb_best_params).fit(X_control, y_control)
clf_treatment = xgb.XGBClassifier(objective="binary:logistic", **xgb_best_params).fit(X_treatment, y_treatment)

predict_test_control = clf_control.predict_proba(X_test)[:, 1]
predict_test_treatment = clf_treatment.predict_proba(X_test)[:, 1]
predict_test_uplift = predict_test_treatment - predict_test_control
df_submission = pd.DataFrame({'uplift': predict_test_uplift}, index=X_test.index)
df_submission.to_csv('all_features_xgb_tuning.csv')

In [17]:
baseline_score = 0.0585
all_features_score = 0.0566
tuning_score = 0.0768


uplift_holdout_baseline_cols = 0.05122098384721485
uplift_1_kfold_baseline_cols = 0.04775477592059265
uplift_6_kfolds_baseline_cols = 0.04856602992351957

uplift_holdout_all_features = 0.05746852762817389
uplift_1_kfold_all_features = 0.05989159396417358
uplift_6_kfolds_all_features = 0.058104041182405114

holdout_diff = uplift_holdout_all_features - uplift_holdout_baseline_cols
kfold_diff = uplift_1_kfold_all_features - uplift_1_kfold_baseline_cols
kfolds_diff = uplift_6_kfolds_all_features - uplift_6_kfolds_baseline_cols

real_diff = all_features_score - baseline_score

print("All features - baseline:")
print(f"holdout diff: {holdout_diff}")
print(f"1 kfold diff: {kfold_diff}")
print(f"6 kfolds diff: {kfolds_diff}")
print(f"real diff: {real_diff}\n")


uplift_holdout_all_features_tuning = 0.08186243440548335
uplift_1_kfold_all_features_tuning = 0.05439141043710725
uplift_6_kfolds_all_features_tuning = 0.06014021157990412

holdout_diff = uplift_holdout_all_features_tuning - uplift_holdout_all_features
kfold_diff = uplift_1_kfold_all_features_tuning - uplift_1_kfold_all_features
kfolds_diff = uplift_6_kfolds_all_features_tuning - uplift_6_kfolds_all_features


real_diff = tuning_score - all_features_score

print("Tuning - no tuning:")
print(f"holdout diff: {holdout_diff}")
print(f"1 kfold diff: {kfold_diff}")
print(f"6 kfolds diff: {kfolds_diff}")
print(f"real diff: {real_diff}")

All features - baseline:
holdout diff: 0.006247543780959042
1 kfold diff: 0.012136818043580931
6 kfolds diff: 0.009538011258885544
real diff: -0.0019000000000000059

Tuning - no tuning:
holdout diff: 0.02439390677730946
1 kfold diff: -0.005500183527066334
6 kfolds diff: 0.002036170397499003
real diff: 0.020199999999999996
